In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, log_loss, confusion_matrix
)
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
import lightgbm as lgb

# ==========================================
# 1. 基础配置：定义列索引（基于Excel结构）
# ==========================================
cat_indices = [1,3,4,5,6,7,8,9]        # 分类变量索引
cont_indices = [2]                     # 连续变量索引 
item_indices = list(range(10, 80))     # 问卷题目索引 

# 目标变量索引 (原始连续分，后续将转为分类标签)
target_index = 91 

# 常用集合
demographics = cat_indices + cont_indices # 所有人口学
all_items = item_indices                  # 所有问卷题

# ==========================================
# 2. 模型特征配置
# ==========================================
model_feature_config = {
    'LASSO_LR': [2,71,80,72,73,76,78,79,24,29,30,35,37,
                12,43,14,16,18,62,47,65,67,68],           
    'SVM': [16,43,37,68,2,76,75,80,24,7,62,21,72,78,57,
                20,67,18],                                
    'XGBoost': [62,80,76,68,67,16,73,65,60,72,75,78,43,
               77,47,71,79,70,37,23,21,64,22,50,12,74,
               61,18,63,14,19,28],                        
    'LightGBM': [76,80,62,16,43,2,72,68],                 
    'Random Forest': [80,76,68,79,78,77,16,67,72,43,60,
                     62,75,70,71,73,50,47,64,12,2,65]
}

In [ ]:
# ==========================================
# 3. 数据读取与智能清洗
# ==========================================
file_path = "内部验证数据集.xlsx"
df = pd.read_excel(file_path, header=2)

if target_index < len(df.columns):
    df = df.dropna(subset=[df.columns[target_index]])
else:
    raise ValueError(f"目标列索引 {target_index} 超出了数据列数范围 (总列数: {len(df.columns)})")

cat_names = [df.columns[i] for i in cat_indices if i < len(df.columns)]
cont_names = [df.columns[i] for i in cont_indices if i < len(df.columns)]
item_names = [df.columns[i] for i in item_indices if i < len(df.columns)]
all_cont_names = cont_names + item_names # 所有需要转为数字的列

for col in all_cont_names:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
# ==========================================
# 4. 准备建模数据
# ==========================================
all_needed_indices = set()
for indices in model_feature_config.values():
    all_needed_indices.update(indices)
all_needed_indices.add(target_index)

# 将索引转为列名
all_needed_cols = [df.columns[i] for i in all_needed_indices if i < len(df.columns)]

# 剔除缺失值 (仅检查需要的列)
df_clean = df.dropna(subset=[df.columns[target_index]]).copy()
print(f"原始样本量: {len(df)}")
print(f"清洗后有效样本量 (仅剔除无标签样本): {len(df_clean)}")

# 4.1 如果样本量为0，打印错误信息
if len(df_clean) == 0:
    print("\n【严重错误】有效样本量为0！请检查以下缺失值情况：")
    print("(显示缺失值最多的前10个列)")
    na_counts = df[all_needed_cols].isna().sum().sort_values(ascending=False)
    print(na_counts.head(10))
    raise ValueError("数据清洗后没有剩余样本，请检查上述列的数据质量或索引配置。")

# 1. 划分索引 (Train/Test)
train_idx, test_idx = train_test_split(df_clean.index, test_size=0.2, random_state=42)

# 2. 提取原始得分用于计算分类阈值
y_score_train = df_clean.loc[train_idx, df.columns[target_index]].astype(float)
y_score_test = df_clean.loc[test_idx, df.columns[target_index]].astype(float)

# 3. 在训练集上计算阈值
q1 = y_score_train.quantile(1/3)
q2 = y_score_train.quantile(2/3)
bins = [-np.inf, q1, q2, np.inf]

print(f"Label cut thresholds (calculated on Train only): Q1={q1:.2f}, Q2={q2:.2f}")

# 4. 应用阈值生成标签 (0:Low, 1:Medium, 2:High)
y_train = pd.cut(y_score_train, bins=bins, labels=[0, 1, 2], include_lowest=True).astype(int)
y_test = pd.cut(y_score_test, bins=bins, labels=[0, 1, 2], include_lowest=True).astype(int)

print("Train class counts:", y_train.value_counts().sort_index().values)
print("Test class counts: ", y_test.value_counts().sort_index().values)

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names")

In [ ]:
# ==========================================
# 5. 网格搜索与模型训练
# ==========================================
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score
# ==========================================

models = {
    'LASSO_LR': LogisticRegression(solver='saga', penalty='elasticnet', max_iter=10000, random_state=42),
    
    'SVM': SVC(probability=True, random_state=42),
    
    'Random Forest': RandomForestClassifier(random_state=42),
    
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42),
    
    'LightGBM': lgb.LGBMClassifier(random_state=42, verbose=-1)
}
# 5.1 定义超参数网格
param_grids = {
    'LASSO_LR': {
        'classifier__l1_ratio': [0.1, 0.5, 0.9], 
        'classifier__C': [0.01, 0.1, 1, 4, 10] 
    },
    
    'Random Forest': {
        'classifier__n_estimators': [100, 500, 1000], 
        'classifier__max_features': ['sqrt', None], 
        'classifier__max_depth': [None, 10, 20],   
        'classifier__min_samples_split': [2, 5]
    },
    
    'XGBoost': {
        'classifier__max_depth': [3, 6, 11], 
        'classifier__learning_rate': [0.01, 0.1, 0.3], 
        'classifier__n_estimators': [200,300,500, 1000],
        'classifier__reg_alpha': [0, 0.1, 1],     
        'classifier__reg_lambda': [1, 5]          
    },
    
    'SVM': {
        'classifier__C': [0.1, 1, 4, 10], 
        'classifier__gamma': ['scale', 0.135],   
        'classifier__kernel': ['rbf']            
    },
    
    'LightGBM': {
        'classifier__learning_rate': [0.01, 0.05],
        'classifier__n_estimators': [500, 1000],
        'classifier__num_leaves': [31, 50],
        'classifier__reg_alpha': [0.1, 1]
    }
}

# 5.2 定义优化目标：Macro-AUC (One-vs-Rest)
scoring_metric = 'roc_auc_ovr'

# 存储最佳模型
best_models = {} 

print(f"\n开始 5折交叉验证 Grid Search (优化目标: {scoring_metric})...")
print("-" * 80)

for name, model_base in models.items():
    print(f"正在优化模型: {name} ...")
    
    # --- A. 准备数据特征 ---
    feature_indices = model_feature_config[name]
    valid_indices = [i for i in feature_indices if i < len(df.columns)]
    feature_names = [df.columns[i] for i in valid_indices]
    
    current_cat = [f for f in feature_names if f in cat_names]
    current_cont = [f for f in feature_names if f not in cat_names]
    
    X_train_curr = df_clean.loc[train_idx, feature_names]
    
    # --- B. 构建 Pipeline ---
    transformers = []
    if current_cont:
        transformers.append(('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')), 
            ('scaler', StandardScaler())
        ]), current_cont))
    if current_cat:
        transformers.append(('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')), 
            ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), current_cat))
        
    pipeline = Pipeline([
        ('preprocessor', ColumnTransformer(transformers=transformers)),
        ('classifier', model_base)
    ])
    
    # --- C. 执行网格搜索 ---
    current_grid = param_grids.get(name, {})
    
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=current_grid,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring=scoring_metric, 
        n_jobs=-1,
        verbose=0
    )
    
    # 训练
    grid.fit(X_train_curr, y_train)
    
    # --- D. 保存与记录 ---
    best_models[name] = grid.best_estimator_
    
    print(f"  > 最佳 AUC (CV平均): {grid.best_score_:.4f}")
    print(f"  > 最佳参数: {grid.best_params_}")
    
    # 简单测试
    X_test_curr = df_clean.loc[test_idx, feature_names]
    y_test_pred = grid.best_estimator_.predict(X_test_curr)
    test_acc = accuracy_score(y_test, y_test_pred)
    print(f"  > 独立测试集 Accuracy: {test_acc:.4f}")
    print("-" * 80)

print("\n网格搜索完成！最佳模型已保存至 best_models 字典中。")

In [ ]:
# ==========================================
# 7. 内部验证：含 95% CI 并按 AUC 排序
# ==========================================
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, cohen_kappa_score, confusion_matrix
from sklearn.preprocessing import label_binarize

# --- 配置 ---
N_BOOTSTRAPS = 1000 
RNG_SEED = 42

# --- 辅助函数 ---
def format_ci(score, lower, upper):
    return f"{score:.3f} ({lower:.3f}-{upper:.3f})"

def compute_metrics_once(y_true, y_pred, y_prob, n_classes):
    # 1. 基础指标
    if n_classes > 2:
        auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
        f1 = f1_score(y_true, y_pred, average='macro')
    else:
        y_prob_pos = y_prob[:, 1]
        auc = roc_auc_score(y_true, y_prob_pos)
        f1 = f1_score(y_true, y_pred)
        
    acc = accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    
    # 2. Brier Score
    y_true_onehot = label_binarize(y_true, classes=range(n_classes))
    if n_classes == 2 and y_true_onehot.shape[1] == 1:
        y_true_onehot = np.hstack([1 - y_true_onehot, y_true_onehot])
    brier = np.mean(np.sum((y_prob - y_true_onehot) ** 2, axis=1))
    
    # 3. Sens/Spec/PPV/NPV
    cm = confusion_matrix(y_true, y_pred, labels=range(n_classes))
    sens_list, spec_list, ppv_list, npv_list = [], [], [], []
    for i in range(n_classes):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - tp - fp - fn
        
        sens_list.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
        spec_list.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        ppv_list.append(tp / (tp + fp) if (tp + fp) > 0 else 0)
        npv_list.append(tn / (tn + fn) if (tn + fn) > 0 else 0)
        
    return {
        'AUC': auc, 'Brier Score': brier, 'Accuracy': acc, 'F1-Score': f1, 'Kappa': kappa,
        'Sensitivity': np.mean(sens_list), 'Specificity': np.mean(spec_list),
        'PPV': np.mean(ppv_list), 'NPV': np.mean(npv_list)
    }

# --- 主计算循环 ---
final_results = []
y_test_arr = np.array(y_test)
n_classes = len(np.unique(y_test_arr))

print(f"开始计算 Bootstrap 95% CI (重采样 {N_BOOTSTRAPS} 次)...")
print("-" * 80)

for name, model in best_models.items():
    print(f"正在评估: {name} ...")
    
    # 准备数据
    feature_indices = model_feature_config[name]
    valid_indices = [i for i in feature_indices if i < len(df.columns)]
    feature_names = [df.columns[i] for i in valid_indices]
    X_test_curr = df_clean.loc[test_idx, feature_names]
    
    # 预测
    y_pred_full = model.predict(X_test_curr)
    y_prob_full = model.predict_proba(X_test_curr)
    
    # 点估计
    original_metrics = compute_metrics_once(y_test_arr, y_pred_full, y_prob_full, n_classes)
    
    # Bootstrap
    boot_metrics = {k: [] for k in original_metrics.keys()}
    for i in range(N_BOOTSTRAPS):
        indices = resample(np.arange(len(y_test_arr)), replace=True, random_state=i)
        if len(np.unique(y_test_arr[indices])) < n_classes: continue 
        
        m = compute_metrics_once(y_test_arr[indices], y_pred_full[indices], y_prob_full[indices], n_classes)
        for k, v in m.items():
            boot_metrics[k].append(v)
            
    # 计算 CI
    res = {'Model': name}
    for k, v in boot_metrics.items():
        lower = np.percentile(v, 2.5)
        upper = np.percentile(v, 97.5)
        res[k] = format_ci(original_metrics[k], lower, upper)
    final_results.append(res)

# --- 生成表格并按 AUC 排序 ---
df_final_ci = pd.DataFrame(final_results)

# 提取 AUC 括号前的数值进行排序
df_final_ci['AUC_Value'] = df_final_ci['AUC'].apply(lambda x: float(x.split(' ')[0]))
df_final_ci = df_final_ci.sort_values(by='AUC_Value', ascending=False).drop(columns=['AUC_Value'])

print("\n=== 内部验证最终结果 (按 AUC 降序) ===")
display(df_final_ci)

# 导出
df_final_ci.to_excel("Table_Internal_Validation_Sorted.xlsx", index=False)
print("表格已保存至本地。")

In [ ]:
# ==========================================
# 7. 内部验证：Calibration & DCA
# ==========================================
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
import numpy as np

# --- A. 绘图设置 ---
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# 类别定义
CLASS_NAMES = {0: "Low Trust", 1: "Medium Trust", 2: "High Trust"}
colors = ['#BD3C29', '#0072B2', '#E69F00', '#009E73', '#CC79A7'] 

# --- B. 定义 DCA 计算函数 ---
def calculate_net_benefit(y_true, y_prob, thresholds):
    net_benefits = []
    n = len(y_true)
    for pt in thresholds:
        pred_pos = y_prob >= pt
        tp = np.sum((y_true == 1) & (pred_pos))
        fp = np.sum((y_true == 0) & (pred_pos))
        if pt == 1.0:
            nb = -np.inf
        else:
            weight = pt / (1 - pt)
            nb = (tp / n) - (fp / n) * weight
        net_benefits.append(nb)
    return np.array(net_benefits)

# --- C. 循环绘制内部验证图 ---
print("开始绘制【内部验证】(Derivation Cohort) 的图表...")

# 获取排序后的模型名
if 'df_final_ci' in locals() and 'Model' in df_final_ci.columns:
    sorted_model_names = df_final_ci['Model'].tolist()
else:
    sorted_model_names = list(best_models.keys())

for class_id, class_label in CLASS_NAMES.items():
    print(f"正在绘制: {class_label} ...")
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), dpi=300)
    
    # 子图 1: Calibration Plot
    axes[0].plot([0, 1], [0, 1], "k:", label="Perfectly Calibrated", linewidth=1.5)
    y_test_binary = (y_test == class_id).astype(int)
    
    for i, name in enumerate(sorted_model_names):
        if name not in best_models: continue
        model = best_models[name]
        color = colors[i % len(colors)]
        
        feature_indices = model_feature_config[name]
        valid_indices = [idx for idx in feature_indices if idx < len(df.columns)]
        feature_names = [df.columns[idx] for idx in valid_indices]
        X_curr = df_clean.loc[test_idx, feature_names]
        
        y_prob = model.predict_proba(X_curr)[:, class_id]
        frac_pos, mean_pred = calibration_curve(y_test_binary, y_prob, n_bins=10)
        
        axes[0].plot(mean_pred, frac_pos, "s-", label=name, color=color, linewidth=2, markersize=5, alpha=0.9)
    
    axes[0].set_title(f"Calibration Plot: {class_label}\n(Derivation Cohort)", fontweight='bold')
    axes[0].set_xlabel("Predicted Probability", fontweight='bold')
    axes[0].set_ylabel("Observed Probability", fontweight='bold')
    axes[0].legend(loc="upper left")
    axes[0].text(0.6, 0.1, f"N = {len(y_test)}", transform=axes[0].transAxes, 
                 fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    # 子图 2: DCA Plot
    dca_thresholds = np.linspace(0.01, 0.99, 100)
    all_pos_rate = np.mean(y_test_binary)
    nb_all = all_pos_rate - (1 - all_pos_rate) * (dca_thresholds / (1 - dca_thresholds))
    nb_none = np.zeros_like(dca_thresholds)
    
    axes[1].plot(dca_thresholds, nb_all, "k:", label="Treat All", linewidth=1.5, alpha=0.7)
    axes[1].plot(dca_thresholds, nb_none, "k--", label="Treat None", linewidth=1.5, alpha=0.7)
    
    for i, name in enumerate(sorted_model_names):
        if name not in best_models: continue
        model = best_models[name]
        color = colors[i % len(colors)]
        
        feature_indices = model_feature_config[name]
        valid_indices = [idx for idx in feature_indices if idx < len(df.columns)]
        feature_names = [df.columns[idx] for idx in valid_indices]
        X_curr = df_clean.loc[test_idx, feature_names]
        
        y_prob = model.predict_proba(X_curr)[:, class_id]
        nb_model = calculate_net_benefit(y_test_binary, y_prob, dca_thresholds)
        axes[1].plot(dca_thresholds, nb_model, "-", label=name, color=color, linewidth=2.5, alpha=0.9)
    
    axes[1].set_title(f"Decision Curve: {class_label}\n(Derivation Cohort)", fontweight='bold')
    axes[1].set_xlabel("Threshold Probability", fontweight='bold')
    axes[1].set_ylabel("Net Benefit", fontweight='bold')
    axes[1].legend(loc="upper right")
    axes[1].set_xlim([0, 0.8])
    y_max = max(np.max(nb_all[(dca_thresholds < 0.8)]), 0.2) * 1.1 
    axes[1].set_ylim([-0.05, y_max])
    
    plt.tight_layout()
    plt.savefig(f"Internal_Fig_{class_label.replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
    plt.savefig(f"Internal_Fig_{class_label.replace(' ', '_')}.pdf", bbox_inches='tight')
    plt.show()

print("内部验证图表绘制完成！")

In [ ]:
# ==========================================
# 9. 外部验证 (External Validation)
# ==========================================
EXTERNAL_FILE_PATH = "外部验证数据集.xlsx" 

print(f"\n正在加载外部验证数据: {EXTERNAL_FILE_PATH} ...")

try:
    # 1. 加载数据
    df_ext = pd.read_excel(EXTERNAL_FILE_PATH, header=2)
    
    # 2. 预处理目标变量
    if target_index < len(df_ext.columns):
        df_ext = df_ext.dropna(subset=[df_ext.columns[target_index]]).copy()
        # 处理连续变量
        for col in cont_names: 
            if col in df_ext.columns:
                df_ext[col] = pd.to_numeric(df_ext[col], errors='coerce')
        
        # 生成标签
        y_ext_score = df_ext.iloc[:, target_index].astype(float)
        y_ext = pd.cut(y_ext_score, bins=bins, labels=[0, 1, 2], include_lowest=True).astype(int)
        
        print(f"外部验证集有效样本量: {len(df_ext)}")
        print(f"类别分布: {y_ext.value_counts().sort_index().to_dict()}")
        
        # 3. 评估循环
        ext_results_list = []
        y_ext_arr = np.array(y_ext)
        n_classes_ext = len(np.unique(y_ext_arr))
        
        print(f"\n开始评估外部验证集 (Bootstrap {N_BOOTSTRAPS} 次)...")
        
        for name, model in best_models.items():
            print(f"正在验证: {name} ...")
            
            # 提取对应特征
            feature_indices = model_feature_config[name]
            valid_indices = [i for i in feature_indices if i < len(df_ext.columns)]
            feature_names = [df_ext.columns[i] for i in valid_indices]
            X_ext_curr = df_ext[feature_names]
            
            # 预测 
            y_pred_ext = model.predict(X_ext_curr)
            y_prob_ext = model.predict_proba(X_ext_curr)
            
            # 计算指标
            # 点估计
            orig_metrics = compute_metrics_once(y_ext_arr, y_pred_ext, y_prob_ext, n_classes_ext)
            
            # Bootstrap CI
            boot_metrics = {k: [] for k in orig_metrics.keys()}
            for i in range(N_BOOTSTRAPS):
                idx = resample(np.arange(len(y_ext_arr)), replace=True, random_state=i)
                if len(np.unique(y_ext_arr[idx])) < n_classes_ext: continue
                
                m = compute_metrics_once(y_ext_arr[idx], y_pred_ext[idx], y_prob_ext[idx], n_classes_ext)
                for k, v in m.items():
                    boot_metrics[k].append(v)
            
            # 格式化结果
            res = {'Model': name}
            for k, v in boot_metrics.items():
                lower = np.percentile(v, 2.5)
                upper = np.percentile(v, 97.5)
                res[k] = format_ci(orig_metrics[k], lower, upper)
            ext_results_list.append(res)
            
        # 4. 生成外部验证表格
        df_ext_results = pd.DataFrame(ext_results_list)
        
        # 同样按 AUC 排序
        df_ext_results['AUC_Sort'] = df_ext_results['AUC'].apply(lambda x: float(x.split(' ')[0]))
        df_ext_results = df_ext_results.sort_values('AUC_Sort', ascending=False).drop('AUC_Sort', axis=1)
        
        print("\n=== 外部验证最终结果 (External Validation) ===")
        display(df_ext_results)
        
        # 导出
        df_ext_results.to_excel("Table_External_Validation_Sorted.xlsx", index=False)
        
    else:
        print("错误：外部数据集中找不到目标列，请检查列索引是否一致。")

except FileNotFoundError:
    print(f"错误：找不到文件 {EXTERNAL_FILE_PATH}，请上传或修改路径。")
except Exception as e:
    print(f"发生错误: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# ==========================================
# 10. 外部验证可视化：Calibration & DCA
# ==========================================
if 'df_ext' not in locals() or 'y_ext' not in locals():
    print("错误：请先运行 Step 9 生成外部验证数据 df_ext！")
else:
    print("开始绘制【外部验证】(External Validation Cohort) 的图表...")
    
    # 获取排序
    if 'df_ext_results' in locals() and 'Model' in df_ext_results.columns:
        sorted_model_names = df_ext_results['Model'].tolist()
    else:
        sorted_model_names = list(best_models.keys())
        
    for class_id, class_label in CLASS_NAMES.items():
        print(f"正在绘制: {class_label} ...")
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6), dpi=300)
        
        # 子图 1: External Calibration
        axes[0].plot([0, 1], [0, 1], "k:", label="Perfectly Calibrated", linewidth=1.5)
        y_ext_arr = np.array(y_ext)
        y_ext_binary = (y_ext_arr == class_id).astype(int)
        
        for i, name in enumerate(sorted_model_names):
            if name not in best_models: continue
            model = best_models[name]
            color = colors[i % len(colors)]
            
            feature_indices = model_feature_config[name]
            valid_indices = [idx for idx in feature_indices if idx < len(df_ext.columns)]
            feature_names = [df_ext.columns[idx] for idx in valid_indices]
            X_ext_curr = df_ext[feature_names] # 注意：这里用 df_ext
            
            y_prob = model.predict_proba(X_ext_curr)[:, class_id]
            frac_pos, mean_pred = calibration_curve(y_ext_binary, y_prob, n_bins=10)
            
            axes[0].plot(mean_pred, frac_pos, "s-", label=name, color=color, linewidth=2, markersize=5, alpha=0.9)
        
        axes[0].set_title(f"Calibration Plot: {class_label}\n(External Validation Cohort)", fontweight='bold')
        axes[0].set_xlabel("Predicted Probability", fontweight='bold')
        axes[0].set_ylabel("Observed Probability", fontweight='bold')
        axes[0].legend(loc="upper left")
        
        axes[0].text(0.6, 0.1, f"N = {len(y_ext)}", transform=axes[0].transAxes, 
                     fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

        # 子图 2: External DCA
        dca_thresholds = np.linspace(0.01, 0.99, 100)
        all_pos_rate = np.mean(y_ext_binary)
        nb_all = all_pos_rate - (1 - all_pos_rate) * (dca_thresholds / (1 - dca_thresholds))
        nb_none = np.zeros_like(dca_thresholds)
        
        axes[1].plot(dca_thresholds, nb_all, "k:", label="Treat All", linewidth=1.5, alpha=0.7)
        axes[1].plot(dca_thresholds, nb_none, "k--", label="Treat None", linewidth=1.5, alpha=0.7)
        
        for i, name in enumerate(sorted_model_names):
            if name not in best_models: continue
            model = best_models[name]
            color = colors[i % len(colors)]
            
            feature_indices = model_feature_config[name]
            valid_indices = [idx for idx in feature_indices if idx < len(df_ext.columns)]
            feature_names = [df_ext.columns[idx] for idx in valid_indices]
            X_ext_curr = df_ext[feature_names]
            
            y_prob = model.predict_proba(X_ext_curr)[:, class_id]
            nb_model = calculate_net_benefit(y_ext_binary, y_prob, dca_thresholds)
            axes[1].plot(dca_thresholds, nb_model, "-", label=name, color=color, linewidth=2.5, alpha=0.9)
        
        axes[1].set_title(f"Decision Curve: {class_label}\n(External Validation Cohort)", fontweight='bold')
        axes[1].set_xlabel("Threshold Probability", fontweight='bold')
        axes[1].set_ylabel("Net Benefit", fontweight='bold')
        axes[1].legend(loc="upper right")
        axes[1].set_xlim([0, 0.8])
        y_max = max(np.max(nb_all[(dca_thresholds < 0.8)]), 0.2) * 1.1 
        axes[1].set_ylim([-0.05, y_max])
        
        plt.tight_layout()
        plt.savefig(f"External_Fig_{class_label.replace(' ', '_')}.png", dpi=300, bbox_inches='tight')
        plt.savefig(f"External_Fig_{class_label.replace(' ', '_')}.pdf", bbox_inches='tight')
        plt.show()
    
    print("外部验证图表绘制完成！")

In [ ]:
# ==============================================================================
# 11. DCA净获益面积图
# ==============================================================================
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from itertools import combinations
import warnings

warnings.filterwarnings("ignore")
COLORS = {'blue': '#6A76C4', 'red': '#CF5B89'} 
BOOTSTRAP_ROUNDS = 1000   
SIG_THRESHOLD = 0.05      
EXTERNAL_FILE_PATH = "外部验证集.xlsx"
CLASS_NAMES = {0: 'Low Trust', 1: 'Medium Trust', 2: 'High Trust'}

# --- 1. 环境准备 ---
print(f"初始化环境 (Bootstrap={BOOTSTRAP_ROUNDS}, Threshold={SIG_THRESHOLD})...")
target_index = 91

if 'y_ext' in locals():
    y_target_data = y_ext
    print("   [Info] 使用内存中的 y_ext。")
else:
    try:
        if 'df_ext' not in locals():
            df_ext = pd.read_excel(EXTERNAL_FILE_PATH, header=2)
        raw_vals = df_ext.iloc[:, target_index].dropna().astype(float)
        if 'bins' not in locals():
            q1, q2 = np.percentile(raw_vals, [33.33, 66.67])
            bins = [-np.inf, q1, q2, np.inf]
        y_target_data = pd.cut(df_ext.iloc[:, target_index], bins=bins, labels=[0, 1, 2], include_lowest=True).astype(int)
        print("   [Info] 自动加载并重建 y_ext 成功。")
    except:
        print("   [Error] 无法加载数据，使用模拟数据演示。")
        y_target_data = np.random.randint(0, 3, 100)

y_ext_raw = np.array(y_target_data)

# --- 2. 核心计算函数 ---

def calculate_net_benefit_curve(y_true, y_prob):
    y_true = np.array(y_true); y_prob = np.array(y_prob)
    thresholds = np.arange(0.01, 1.0, 0.01)
    n = len(y_true)
    nb_values = []
    for thresh in thresholds:
        tp = np.sum((y_prob >= thresh) & (y_true == 1))
        fp = np.sum((y_prob >= thresh) & (y_true == 0))
        weight = thresh / (1 - thresh)
        nb = (tp / n) - (fp / n) * weight
        nb_values.append(nb)
    return np.array(nb_values)

def calculate_areas(nb_values):
    """计算正负面积"""
    pos_curve = np.maximum(nb_values, 0)
    pos_area = np.trapz(pos_curve, dx=0.01)
    neg_curve = np.minimum(nb_values, 0)
    neg_area = abs(np.trapz(neg_curve, dx=0.01))
    return pos_area, neg_area

def calculate_treat_all_pos_area(y_true):
    """计算基准线面积"""
    y_true = np.array(y_true)
    n = len(y_true)
    prevalence = np.sum(y_true) / n
    thresholds = np.arange(0.01, 1.0, 0.01)
    nb_values = []
    for thresh in thresholds:
        weight = thresh / (1 - thresh)
        nb = prevalence - (1 - prevalence) * weight
        nb_values.append(nb)
    pos_curve = np.maximum(np.array(nb_values), 0)
    return np.trapz(pos_curve, dx=0.01)

def calculate_pairwise_raw_p_values(model_data, y_true, n_boot=100):
    """计算原始 P 值 (Bootstrap)"""
    models = list(model_data.keys())
    rng = np.random.default_rng(42)
    n = len(y_true)
    
    obs_areas = {}
    for m, p in model_data.items():
        nb = calculate_net_benefit_curve(y_true, p)
        pos, _ = calculate_areas(nb)
        obs_areas[m] = pos
        
    pairs = list(combinations(models, 2))
    obs_diffs = {pair: obs_areas[pair[0]] - obs_areas[pair[1]] for pair in pairs}
    
    boot_diffs = {pair: [] for pair in pairs}
    for _ in range(n_boot):
        indices = rng.choice(n, n, replace=True)
        y_boot = y_true[indices]
        for pair in pairs:
            m1, m2 = pair
            p1 = model_data[m1][indices]
            p2 = model_data[m2][indices]
            nb1 = calculate_net_benefit_curve(y_boot, p1)
            nb2 = calculate_net_benefit_curve(y_boot, p2)
            d1, _ = calculate_areas(nb1)
            d2, _ = calculate_areas(nb2)
            boot_diffs[pair].append(d1 - d2)
            
    raw_p_values = {}
    for pair, diff_list in boot_diffs.items():
        obs = obs_diffs[pair]
        p = np.mean(np.abs(np.array(diff_list)) >= np.abs(obs))
        if p == 0: p = 1 / (n_boot + 1)
        raw_p_values[pair] = p
    return raw_p_values

def holm_bonferroni_correction(p_values_dict):
    """Holm-Bonferroni"""
    if not p_values_dict: return {}
    pairs = list(p_values_dict.keys())
    raw_p = [p_values_dict[k] for k in pairs]
    sorted_indices = np.argsort(raw_p)
    m = len(raw_p)
    adj_p = np.zeros(m)
    current_max = 0
    for i in range(m):
        idx = sorted_indices[i]
        p = raw_p[idx]
        corrected = p * (m - i)
        corrected = max(corrected, current_max)
        current_max = corrected
        adj_p[idx] = min(1.0, corrected)
    result = {}
    for i, pair in enumerate(pairs):
        result[pair] = adj_p[i]
    return result

# --- 3. 绘图执行 ---
print(f"启动全套分析 (Alpha={SIG_THRESHOLD} | Correction=Holm | Boot={BOOTSTRAP_ROUNDS})...")

unique_classes = np.sort(np.unique(y_ext_raw))
labels_map = CLASS_NAMES

if 'best_models' not in locals():
    print("错误: best_models 未定义。")
else:
    for target_class in unique_classes:
        class_label = labels_map.get(target_class, f"Class {target_class}")
        clean_name = class_label.replace(" ", "_")
        print(f"\n🔵 [Processing] {class_label}")
        
        y_binary = (y_ext_raw == target_class).astype(int)
        baseline_pos_area = calculate_treat_all_pos_area(y_binary)
        
        current_models = {}    
        plot_data = [] 
        
        # --- A. 准备绘图数据 ---
        for m_name, pipeline in best_models.items():
            try:
                if 'model_feature_config' in locals() and m_name in model_feature_config:
                    feature_indices = model_feature_config[m_name]
                    if hasattr(df_ext, 'iloc'):
                         valid_indices = [i for i in feature_indices if i < df_ext.shape[1]]
                         X_curr = df_ext.iloc[:, valid_indices]
                    else:
                         X_curr = np.zeros((len(y_binary), 1))

                    if hasattr(pipeline, "predict_proba"):
                        probs = pipeline.predict_proba(X_curr)
                        p_target = probs[:, target_class]
                        
                        nb_curve = calculate_net_benefit_curve(y_binary, p_target)
                        pos_area, neg_area = calculate_areas(nb_curve)
                        
                        height = pos_area
                        delta = height - baseline_pos_area
                        color = COLORS['blue'] if pos_area > neg_area else COLORS['red']
                        
                        plot_data.append({'name': m_name, 'height': height, 'color': color, 'delta': delta})
                        current_models[m_name] = p_target
            except: pass
        
        if not plot_data: continue

        # --- B. 计算 P 值 (Holm-Bonferroni) ---
        print(f"   运行 {BOOTSTRAP_ROUNDS} 次模拟 (校正: Holm-Bonferroni)...")
        raw_p_dict = calculate_pairwise_raw_p_values(current_models, y_binary, n_boot=BOOTSTRAP_ROUNDS)
        adj_p_dict = holm_bonferroni_correction(raw_p_dict)
        
        # --- C. 绘制 DCA 柱状图 ---
        plt.figure(figsize=(10, 7), dpi=300)
        names = [d['name'] for d in plot_data]
        heights = [d['height'] for d in plot_data]
        colors = [d['color'] for d in plot_data]
        deltas = [d['delta'] for d in plot_data]
        x_pos = np.arange(len(names))
        
        bars = plt.bar(x_pos, heights, width=0.5, color=colors, edgecolor='black', alpha=0.9, zorder=3)
        
        # 标注 Delta 值
        for bar, d in zip(bars, deltas):
            symbol = '↑' if d > 0 else '↓'
            text_str = f"{symbol} {abs(d):.3f}"
            plt.text(bar.get_x() + bar.get_width()/2, 0.005, text_str, 
                     ha='center', va='bottom', color='black', fontweight='bold', fontsize=10, zorder=5)

        # 动态显著性标注逻辑
        sig_pairs = [(pair, p) for pair, p in adj_p_dict.items() if p < SIG_THRESHOLD]
        
        top_visual = max(heights) if heights else 0.1
        final_top = top_visual * 1.3
        
        if sig_pairs:
            sig_pairs.sort(key=lambda x: x[1]) 
            
            line_start_y = top_visual + 0.02
            step_y = 0.04  
            
            for i, ((m1, m2), p) in enumerate(sig_pairs):
                idx1 = names.index(m1); idx2 = names.index(m2)
                x1, x2 = min(idx1, idx2), max(idx1, idx2)
                y_h = line_start_y + i * step_y
                y_leg = 0.005
                
                plt.plot([x1, x1, x2, x2], [y_h - y_leg, y_h, y_h, y_h - y_leg], lw=1.2, c='black')
                if p < 0.001: star = "***"
                elif p < 0.01: star = "**"
                else: star = "*"  
                
                plt.text((x1 + x2)/2, y_h, star, ha='center', va='bottom', color='black', fontsize=12, fontweight='bold')
            
            final_top = line_start_y + len(sig_pairs) * step_y + 0.05

        plt.ylim(0, final_top)
        plt.title(f"Clinical Utility (Pos Area | Threshold < {SIG_THRESHOLD})\nTarget: {class_label}", fontsize=14, fontweight='bold', pad=15)
        plt.xticks(x_pos, names, fontsize=11)
        plt.ylabel("Area under net benefit curve")
        plt.tight_layout()
        plt.savefig(f"DCA_Bar_{clean_name}.png", dpi=300, bbox_inches='tight')
        plt.show()
        print(f"   [图表 1] DCA柱状图: DCA_Bar_{clean_name}.png")

        # --- D.Adj. P 热力图 ---
        models_list = list(current_models.keys())
        p_matrix = pd.DataFrame(np.nan, index=models_list, columns=models_list)
        np.fill_diagonal(p_matrix.values, 1.0)
        
        for (m1, m2), p in adj_p_dict.items():
            p_matrix.loc[m1, m2] = p
            p_matrix.loc[m2, m1] = p
            
        plt.figure(figsize=(8, 6), dpi=300)
        annot_labels = p_matrix.copy().astype(object)
        
        for i in range(len(models_list)):
            for j in range(len(models_list)):
                val = p_matrix.iloc[i, j]
                if pd.isna(val): continue
                if i == j: 
                    annot_labels.iloc[i, j] = ""
                    continue
                
                # 🎯 动态文本逻辑 (核心修改)
                if val < 0.001:
                    txt = "<.001\n***"
                elif val < 0.01:
                    txt = f"{val:.3f}\n**"
                elif val < 0.05:
                    txt = f"{val:.3f}\n*"
                else:
                    txt = f"{val:.3f}"
                
                annot_labels.iloc[i, j] = txt
        
        sns.heatmap(p_matrix, annot=annot_labels, fmt="", 
                    cmap="Reds_r", vmin=0, vmax=0.1, 
                    cbar_kws={'label': f'Adj. P-value (Holm)'},
                    linewidths=1, linecolor='gray')
        
        plt.title(f"Pairwise Comparison (Holm Adj. P)\nTarget: {class_label}", fontsize=14, fontweight='bold', pad=15)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.savefig(f"Heatmap_AdjP_{clean_name}.png", dpi=300, bbox_inches='tight')
        plt.show()
        print(f"   [图表 2] 热力图: Heatmap_AdjP_{clean_name}.png")

        # --- E. 导出 CSV ---
        csv_data = []
        for (m1, m2), ap in adj_p_dict.items():
            rp = raw_p_dict.get((m1, m2), np.nan)
            csv_data.append({
                "Category": class_label,
                "Model A": m1, "Model B": m2,
                "Raw P-value": rp,
                "Adj. P-value": ap,
                "Significant": "Yes" if ap < SIG_THRESHOLD else "No"
            })
        pd.DataFrame(csv_data).to_csv(f"Stats_Table_{clean_name}.csv", index=False)
        print(f"   [表格 3] 统计数据表: Stats_Table_{clean_name}.csv")

print("\n 全部完成！")

In [ ]:
# ==========================================
# 12.校准指标表格
# ==========================================

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression  

# --- 1. 定义计算函数 ---
def calculate_calibration_slope(y_true, y_prob):
    eps = 1e-15
    p = np.clip(np.asarray(y_prob), eps, 1 - eps)
    x = np.log(p / (1 - p)).reshape(-1, 1)

    lr = LogisticRegression(penalty=None, solver="lbfgs", max_iter=2000)
    lr.fit(x, np.asarray(y_true))
    return float(lr.coef_[0][0])

def get_class_proba(model, X, class_id):
    classes = model.classes_
    proba_col = int(np.where(classes == class_id)[0][0])
    return model.predict_proba(X)[:, proba_col]

def plot_academic_table(df_display, title, filename):
    fig, ax = plt.subplots(figsize=(10, 3 + len(df_display)*0.3), dpi=300)
    ax.axis('off')

    the_table = ax.table(
        cellText=df_display.values,
        colLabels=df_display.columns,
        loc='center',
        cellLoc='center'
    )

    the_table.auto_set_font_size(False)
    the_table.set_fontsize(11)
    the_table.scale(1, 2.0)

    for (row, col), cell in the_table.get_celld().items():
        cell.set_text_props(fontfamily='sans-serif')
        cell.set_facecolor('white')
        cell.set_edgecolor('black')
        cell.set_linewidth(1.0)

        if row == 0:
            cell.set_text_props(weight='bold', color='black')
        else:
            cell.set_text_props(weight='normal', color='black')

    plt.title(title, fontsize=12, fontweight='bold', y=1.02, loc='center')

    plt.savefig(f"{filename}.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{filename}.pdf", bbox_inches='tight')
    plt.show()
    print(f"图片已生成: {filename}.png")


# --- 2. 主执行逻辑 ---
if 'df_ext' in locals() and 'y_ext' in locals():
    print("正在生成校准表格...")

    model_stats = {
        name: {'Slope_Int': [], 'MSE_Int': [], 'Slope_Ext': [], 'MSE_Ext': []}
        for name in best_models.keys()
    }
    results_list = []

    # 1. 遍历类别
    for class_id, class_label in CLASS_NAMES.items():
        print(f"   -> 处理类别: {class_label}")
        y_int_bin = (y_test == class_id).astype(int)
        y_ext_bin = (np.array(y_ext) == class_id).astype(int)

        class_rows = []

        for name, model in best_models.items():
            # 获取数据
            if 'model_feature_config' in globals():
                f_idx = model_feature_config[name]
                valid_idx = [i for i in f_idx if i < len(df.columns)]
                cols = df.columns[valid_idx]
                X_int = df_clean.loc[test_idx, cols]
                X_ext = df_ext[cols]
            else:
                X_int = df_clean.loc[test_idx]
                X_ext = df_ext

            # 概率列按 model.classes_ 对齐
            p_int = get_class_proba(model, X_int, class_id)
            p_ext = get_class_proba(model, X_ext, class_id)

            # calibration slope（logistic）
            s_int = calculate_calibration_slope(y_int_bin, p_int)
            m_int = mean_squared_error(y_int_bin, p_int)
            s_ext = calculate_calibration_slope(y_ext_bin, p_ext)
            m_ext = mean_squared_error(y_ext_bin, p_ext)

            model_stats[name]['Slope_Int'].append(s_int)
            model_stats[name]['MSE_Int'].append(m_int)
            model_stats[name]['Slope_Ext'].append(s_ext)
            model_stats[name]['MSE_Ext'].append(m_ext)

            class_rows.append({
                'Model': name,
                'Slope (Int)': f"{s_int:.3f}",
                'MSE (Int)': f"{m_int:.3f}",
                'Slope (Ext)': f"{s_ext:.3f}",
                'MSE (Ext)': f"{m_ext:.3f}"
            })

            results_list.append({
                'Category': class_label, 'Model': name,
                'Slope_Int': s_int, 'MSE_Int': m_int,
                'Slope_Ext': s_ext, 'MSE_Ext': m_ext
            })

        plot_academic_table(
            pd.DataFrame(class_rows),
            title=f"Table. Calibration Metrics - {class_label}",
            filename=f"Table_Calibration_{class_label.replace(' ', '_')}"
        )

    # 2. 计算 Macro-average
    print("   -> 计算 Macro-average...")
    avg_rows = []
    for name in best_models.keys():
        avg_s_int = float(np.mean(model_stats[name]['Slope_Int']))
        avg_m_int = float(np.mean(model_stats[name]['MSE_Int']))
        avg_s_ext = float(np.mean(model_stats[name]['Slope_Ext']))
        avg_m_ext = float(np.mean(model_stats[name]['MSE_Ext']))

        avg_rows.append({
            'Model': name,
            'Slope (Int)': f"{avg_s_int:.3f}",
            'MSE (Int)': f"{avg_m_int:.3f}",
            'Slope (Ext)': f"{avg_s_ext:.3f}",
            'MSE (Ext)': f"{avg_m_ext:.3f}"
        })

        results_list.append({
            'Category': 'Total Average', 'Model': name,
            'Slope_Int': avg_s_int, 'MSE_Int': avg_m_int,
            'Slope_Ext': avg_s_ext, 'MSE_Ext': avg_m_ext
        })

    plot_academic_table(
        pd.DataFrame(avg_rows),
        title="Table. Calibration Metrics - Total Average (Macro)",
        filename="Table_Calibration_Total_Average"
    )

    # 导出 CSV
    pd.DataFrame(results_list).to_csv("Calibration_Metrics_Academic.csv", index=False)
    print("\n📋 原始数据已保存为 CSV")
else:
    print("未检测到外部验证数据。")
